In [ ]:
# install the below dependencies
# !pip install python-dotenv smolagents[telemetry] langfuse opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

### pre-requisite
- API Keys for
	- Hugging Face (Hugging Face write token)
	- Langfuse (secret key, public key, host url)
- Access to Qwen3 or similar Model on the hugging face platform

In [ ]:
# load env variables to get api keys and other secrets
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
# Setup langfuse client
from langfuse import Langfuse

langfuse = Langfuse(
	secret_key=os.getenv("SECRET_KEY"),
	public_key=os.getenv("PUBLIC_KEY"),
	host=os.getenv("LANGFUSE_HOST"),
)

In [ ]:
# authenticate langfuse client connection
from langfuse import get_client

langfuse = get_client()

# Verify connection, do not use in production as this is a synchronous call
if langfuse.auth_check():
	print("Langfuse client is authenticated and ready!")
else:
	print("Authentication failed. Please check your credentials and host.")

In [ ]:
# set up opentelemetry with langfuse for tracing information
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [ ]:
# set up opentelemetry tracer
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

In [ ]:
# set up smolagents instrumentor
SmolagentsInstrumentor().instrument(trace_provider=trace_provider)

In [ ]:
# setup smolagents agent as Code Agent
from smolagents import CodeAgent, InferenceClientModel, DuckDuckGoSearchTool

agent_langfuse = CodeAgent(
	tools=[DuckDuckGoSearchTool()],
	model=InferenceClientModel(
		model_id="Qwen/Qwen3-Next-80B-A3B-Instruct",  # mention the LLM model id here
		provider="together",
		api_key=os.getenv("HF_WRITE_TOKEN"),  # change provider based on your preference
	),
)

In [ ]:
# prompt for the agent to get learning material for the user and format in the markdown table format
agent_langfuse.run(
	"""
	You are a helpful assistant that recommends books for learning technical topics.
You have access to a search tool called {search_tool}. Use it to find books.

**Step 1:** Use DuckDuckGo to search the web for books relevant to the user's query.
**Step 2:** Parse the search results and filter out non‑books (articles, blogs, etc.).
**Step 3:** Rank the books by relevance and publication year.
**Step 4:** Return a markdown table with columns: Rank, Title, Author, Year, URL.
User's query: please search for materials about creating production ready agentic rag using rust or go language.
	"""
)